# 📘 Disciplina: Programação para Ciência de Dados

## 🧩 Matéria: Introdução ao Jupyter Notebook

### 👥 Integrant- Lucas Gabriel Guedes Limeira
- Maria Fernanda  Pedro  

---

Este notebook foi criado como parte da atividade introdutória da disciplina, com o objetivo de configurar e testar o ambiente Jupyter em contêiner Docker.

A seguir, serão adicionadas células de código e análises referentes ao tema proposto.


In [1]:
# Importações básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_digits, load_iris, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [4]:
# Trocando vírgulas por pontos para que as colunas sejam reconhecidas como números:
data = pd.read_csv("estado-de-residencia-do-declarante.csv", delimiter=';', dtype=str)

for col in data.columns:
    data[col] = data[col].str.replace(',', '.', regex=False)
    data[col] = pd.to_numeric(data[col], errors='ignore')

colunas_numericas = data.select_dtypes(include=['number']).columns.tolist()

print(f"Total de colunas numéricas: {len(colunas_numericas)}")
print("Colunas numéricas detectadas:")
print(colunas_numericas)

Total de colunas numéricas: 42
Colunas numéricas detectadas:
['Ano-Calendário', 'Quantidade de Declarantes', 'Rendimentos Tributáveis', 'Rendimentos sujeitos a Tributação Exclusiva', 'Rendimentos Isentos', 'Dedução Previdenciária Total', 'Deduções Previdência Oficial', 'Deduções Previdenciárias RRA', 'Deduções Previdência Privada e FAPI', 'Deduções com Dependentes', 'Deduções com Instrução', 'Deduções com despesas Médicas', 'Dedução de Livro Caixa', 'Deduções  com Pensão Alimentícia Total', 'Deduções com Pensão Alimentícia Judicial', 'Deduções com Pensão Alimentícia Escritural', 'Deduções com Pensão Alimentícia RRA', 'Desconto Padrão', 'Base de Cálculo (RTL)', 'Imposto Devido', 'Dedução de Incentivo', 'Imposto Devido 1', 'Imposto Devido RRA', 'Imposto Devido Total', 'Imposto Pago', 'Imposto a Pagar', 'Imposto a Restituir', 'Dívidas   e Ônus', 'Doações e Heranças Recebidas', 'Bens e Direitos', 'Previdência Complementar (VGBL)', 'Valores Pagos com Previdência Complementar (PGBL)', 'Valor